In [1]:
# Load Packages
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
import joblib
import pickle
import statistics as stat

In [2]:
# Load the data
dataI = pd.read_csv("final_table.csv")

In [3]:
# Clean dataS
dataI = dataI.drop(columns=['week', 'Arrest', 'Domestic', 'DayofMonth'])
dataI['hour'] = dataI['hour'].astype('category')
dataI['is_holiday'] = dataI['is_holiday'].apply(lambda x: 1 if x > 0 else 0).astype("category")
dataI['is_full_moon'] = dataI['is_full_moon'].apply(lambda x: 1 if x > 0 else 0).astype("category")
dataI['is_day'] = dataI['is_day'].apply(lambda x: 1 if x > 0 else 0).astype("category")

In [4]:
# Indetify features and response variables
X = dataI.iloc[:,:23]
Y = dataI.iloc[:,23:]

In [5]:
# Make categorical features into dummy variables
categorical_features = X.select_dtypes(include=['object', 'category'])
def preprocess_and_create_dummies(data):
    processed_data = data.copy()
    processed_data = pd.get_dummies(processed_data, columns=categorical_features.columns)
    return processed_data
X= X.pipe(preprocess_and_create_dummies)

In [ ]:
# Fit models

num = 1
for col in Y.columns:
    # Extract the response variable
    y = Y[col]
    
    # Train a random forest model
    model = RandomForestRegressor()
    model.fit(X, y)
    
    # Name the model according to the response variable
    model_name = col + "_model.joblib.gz"
    
    # Store the trained model in the list
    joblib.dump(model, model_name, compress=('gzip', 4))

    print(f"Done training model {model_name} - {num}/{len(Y.columns)}")
    num += 1

In [ ]:
# with open("trained_models.pkl", 'rb') as file:
#     trained_models = pickle.load(file)

In [6]:
models_path = ['Primary Type_BURGLARY_model.joblib.gz',
       'Primary Type_CRIM SEXUAL ASSAULT_model.joblib.gz',
       'Primary Type_ROBBERY_model.joblib.gz', 'Primary Type_INTIMIDATION_model.joblib.gz',
       'Primary Type_HOMICIDE_model.joblib.gz', 'Primary Type_KIDNAPPING_model.joblib.gz',
       'Primary Type_HUMAN TRAFFICKING_model.joblib.gz', 'Beat_1111_model.joblib.gz',
       'Beat_1112_model.joblib.gz', 'Beat_1113_model.joblib.gz', 'Beat_1114_model.joblib.gz',
       'Beat_1115_model.joblib.gz', 'Beat_1121_model.joblib.gz', 'Beat_1122_model.joblib.gz',
       'Beat_1123_model.joblib.gz', 'Beat_1124_model.joblib.gz', 'Beat_1125_model.joblib.gz',
       'Beat_1131_model.joblib.gz', 'Beat_1132_model.joblib.gz', 'Beat_1133_model.joblib.gz',
       'Beat_1134_model.joblib.gz', 'Beat_1135_model.joblib.gz', 'n_crimes_model.joblib.gz']
trained_models = []
for file in models_path:
    model = joblib.load(file)
    name = file.replace('.joblib.gz', '')
    trained_models.append((name, model))

In [9]:
calendar = pd.date_range(start='2024-01-01', end='2024-03-31', freq='H')
full_moon_dates = [
    "2024-01-25",
    "2024-02-24",
    "2024-03-25",
    "2024-04-23",
    "2024-05-23",
    "2024-06-21",
    "2024-07-21",
    "2024-08-19",
    "2024-09-17",
    "2024-10-17",
    "2024-11-15",
    "2024-12-15"
]

holiday_dates = [
    "2024-01-01",  # New Year's Day
    "2024-01-15",  # Martin Luther King Jr. Day
    "2024-02-19",  # Washington's Birthday (Presidents Day)
    "2024-05-27",  # Memorial Day
    "2024-07-04",  # Independence Day
    "2024-09-02",  # Labor Day
    "2024-10-14",  # Columbus Day
    "2024-11-11",  # Veterans Day
    "2024-11-28",  # Thanksgiving Day
    "2024-12-25"   # Christmas Day
]

full_calendar = pd.DataFrame({
    'Date': calendar,
    'Year': calendar.year,
    'month': calendar.strftime('%b'),
    'DayOfWeek': calendar.strftime('%a'),  # Monday=0, Sunday=6
    'hour': calendar.hour.astype('category'),
    'Week' : [date.isocalendar()[1] for date in calendar], 
    'is_holiday': [1 if str(date)[:10] in holiday_dates else 0 for date in calendar],  # Check if date is a holiday
    'is_full_moon': [1 if str(date)[:10] in full_moon_dates else 0 for date in calendar]  # Check if date is a full moon
})

In [10]:
sunrise = [7]*31 + [7]*4 + [6]*25 + [6]*9 + [7]*6 + [6]*16
sunset = [16]*27 + [17]*4 + [17]*29 + [17]*9 + [18]*7 + [19]*15
full_calendar['is_day'] = full_calendar['hour'].apply(lambda x: 1 if sunrise[x] <= x < sunset[x] else 0).astype('category')
full_calendar['is_full_moon'] = full_calendar['is_full_moon'].astype('category')
full_calendar['is_holiday'] = full_calendar['is_holiday'].astype('category')

In [11]:
week_concatenated = (full_calendar.groupby('Week')['Date'].agg(['first', 'last']).reset_index())
week_concatenated['last'] = week_concatenated['last'].dt.date # Remove the hours from the 'last' column
week_concatenated['weeks'] = week_concatenated['first'].astype(str) + ' - ' + week_concatenated['last'].astype(str)
full_calendar = full_calendar.merge(week_concatenated[['Week', 'weeks']], on='Week', how='left')

In [12]:
new_obs_t = full_calendar[full_calendar['weeks'] == "2024-01-01 - 2024-01-07"][['Year','month', 'DayOfWeek', 'hour', 'is_holiday', 'is_full_moon', 'is_day']]
new_obs = new_obs_t
new_obs['moonphase'] = 6
new_obs['temperature'] = -122
new_obs['humidity'] = 710
new_obs['feels_like'] = -188
new_obs['rain'] = 0
new_obs['snowfall'] = 0
new_obs['snow_depth'] = 0.9
new_obs['cloud_cover'] = 850
new_obs['wind_speed'] = 205
new_obs['wind_gusts'] = 335
new_obs['shortwave_radiation'] = 0
new_obs['direct_radiation'] = 0
new_obs['dew'] = -176
new_obs['sealevelpressure'] = 10316
new_obs['visibility'] = 156
new_obs['Unemployment'] = 112

In [13]:
categorical_features = new_obs.select_dtypes(include=['object', 'category'])
def preprocess_and_create_dummies(data):
    processed_data = data.copy()
    processed_data = pd.get_dummies(processed_data, columns=categorical_features.columns)
    return processed_data
new_obs= new_obs.pipe(preprocess_and_create_dummies)

# List of columns to add
new_columns = ['month_Apr', 'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul',
               'month_Jun', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct', 'month_Sep']

# Add columns if they don't exist already
for col in new_columns:
    if col not in new_obs.columns:
        new_obs[col] = 0

# Reorder columns
new_obs = new_obs[['Year', 'moonphase', 'temperature', 'humidity', 'feels_like', 'rain',
       'snowfall', 'snow_depth', 'cloud_cover', 'wind_speed', 'wind_gusts',
       'shortwave_radiation', 'direct_radiation', 'dew', 'sealevelpressure',
       'visibility', 'Unemployment', 'month_Apr',
       'month_Aug', 'month_Dec', 'month_Feb', 'month_Jan', 'month_Jul',
       'month_Jun', 'month_Mar', 'month_May', 'month_Nov', 'month_Oct',
       'month_Sep', 'DayOfWeek_Fri', 'DayOfWeek_Mon', 'DayOfWeek_Sat',
       'DayOfWeek_Sun', 'DayOfWeek_Thu', 'DayOfWeek_Tue', 'DayOfWeek_Wed',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'is_holiday_0',
       'is_holiday_1', 'is_full_moon_0', 'is_full_moon_1', 'is_day_0',
       'is_day_1']]

In [14]:
# Select the model from the list based on the crime type
selected_model = None
for model_name, model in trained_models:
    if 'Primary Type_' + 'ROBBERY' + '_model' in model_name:
        selected_model = model
        break

predictions = selected_model.predict(new_obs)

In [15]:
# Put predictions next to Day - hour combination
matrix = np.array(predictions).reshape(7, 24)

In [16]:
fig = px.imshow(matrix,
                labels=dict(x="Hour", y="Day", color="Violent Crimes"),
                y=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
                x=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12',
                   '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23'])

# Update layout
fig.update_layout(title='Violent Crimes Heatmap')
fig.show()


In [17]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61074 entries, 0 to 61073
Data columns (total 66 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Year                 61074 non-null  int64  
 1   moonphase            61074 non-null  float64
 2   temperature          61074 non-null  float64
 3   humidity             61074 non-null  int64  
 4   feels_like           61074 non-null  float64
 5   rain                 61074 non-null  float64
 6   snowfall             61074 non-null  float64
 7   snow_depth           61074 non-null  float64
 8   cloud_cover          61074 non-null  int64  
 9   wind_speed           61074 non-null  float64
 10  wind_gusts           61074 non-null  float64
 11  shortwave_radiation  61074 non-null  float64
 12  direct_radiation     61074 non-null  float64
 13  dew                  61074 non-null  float64
 14  sealevelpressure     61074 non-null  float64
 15  visibility           61074 non-null 